## OSM Preprocessing

In [3]:
!pip install osmnx google-cloud-storage rasterio > /dev/null

!python -m osmnx --version

/usr/bin/python3: No module named osmnx.__main__; 'osmnx' is a package and cannot be directly executed


In [4]:
import os
import geopandas as gpd
import osmnx as ox
import rasterio
from rasterio.features import rasterize
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt


drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Area of Interest(Hamburg) Selection**

In [5]:
# Çalışılacak alan (Area of Interest - AOI)
place_name = "Altona, Hamburg, Germany"

# OSM verileri için kullanılacak etiketler
# Projenin ihtiyacına göre bu etiketler zenginleştirilebilir.
tags = {
    "building": True,
    "highway": True,
    "leisure": ["park", "garden", "playground"],
    "landuse": ["forest", "grass", "greenfield"],
    "natural": ["water", "wetland"]
}


PROCESSED_DIR = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/osm/'
os.makedirs(PROCESSED_DIR, exist_ok=True)
print(f"OSM verileri '{PROCESSED_DIR}' klasörüne kaydedilecek.")


# Referans raster (şimdilik devre dışı)
# REFERENCE_RASTER_PATH = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/sentinel2/B04_10m.jp2'
# if not os.path.exists(REFERENCE_RASTER_PATH):
#     raise FileNotFoundError(f"Referans raster dosyası bulunamadı: {REFERENCE_RASTER_PATH}.")
# print(f"Referans raster olarak '{REFERENCE_RASTER_PATH}' kullanılacak.")


OSM verileri '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/osm/' klasörüne kaydedilecek.


**Downloading OSM Data with selected tags**

In [6]:
# OSM verilerini indirme
print(f"'{place_name}' için OSM verileri indiriliyor...")
gdf = ox.features_from_place(place_name, tags=tags)
print(f"Veri indirme tamamlandı. Toplam {len(gdf)} adet geometri bulundu.")

print(gdf.head())

'Altona, Hamburg, Germany' için OSM verileri indiriliyor...
Veri indirme tamamlandı. Toplam 87058 adet geometri bulundu.
                                geometry crossing            highway  \
element id                                                             
node    131185   POINT (9.8874 53.56527)       no    traffic_signals   
        491108  POINT (9.91555 53.58605)      NaN  motorway_junction   
        491128  POINT (9.89759 53.55653)      NaN  motorway_junction   
        491129  POINT (9.89719 53.55379)      NaN    traffic_signals   
        496093  POINT (9.91162 53.57753)      NaN          milestone   

               traffic_signals:direction TMC:cid_58:tabcd_1:Class  \
element id                                                          
node    131185                   forward                      NaN   
        491108                       NaN                    Point   
        491128                       NaN                    Point   
        491129               

**Separating and Saving Data Layers**

In [8]:
# Binalar
buildings = gdf[gdf['building'].notna()]
print(f"Toplam {len(buildings)} adet bina bulundu.")

# Yollar (Sadece çizgisel geometrileri alalım)
roads = gdf[gdf['highway'].notna() & (gdf.geom_type == 'LineString')]
print(f"Toplam {len(roads)} adet yol segmenti bulundu.")

# Yeşil Alanlar (Parklar, ormanlar, çimenlikler)
green_spaces = gdf[
    (gdf['leisure'].isin(['park', 'garden', 'playground'])) |
    (gdf['landuse'].isin(['forest', 'grass', 'greenfield']))
]
print(f"Toplam {len(green_spaces)} adet yeşil alan bulundu.")

# Su Kütleleri
water = gdf[gdf['natural'].isin(['water', 'wetland'])]
print(f"Toplam {len(water)} adet su kütlesi bulundu.")

Toplam 43677 adet bina bulundu.
Toplam 33479 adet yol segmenti bulundu.
Toplam 1695 adet yeşil alan bulundu.
Toplam 319 adet su kütlesi bulundu.


**Rasterizing Vector layers**

In [ ]:
# Bu adımda, ayırdığımız her bir vektör katmanını, referans Sentinel-2 görüntümüzle
# aynı boyut ve çözünürlükte bir raster (GeoTIFF) dosyasına dönüştüreceğiz. [cite: 151, 173]

def rasterize_and_save(gdf, reference_raster_path, output_path):
    """
    Bir GeoDataFrame'i referans raster'a göre rasterleştirir ve kaydeder.
    """
    # Referans raster'ı aç ve metaverilerini al
    with rasterio.open(reference_raster_path) as src:
        ref_meta = src.meta.copy()
        ref_transform = src.transform
        ref_shape = src.shape
        ref_crs = src.crs

    # Vektörün CRS'sini referans ile aynı yap
    if gdf.crs != ref_crs:
        gdf = gdf.to_crs(ref_crs)

    print(f"'{output_path}' için rasterleştirme başlıyor...")
    # Rasterize işlemi
    rasterized_array = rasterize(
        shapes=[(geom, 1) for geom in gdf.geometry], # Geometrilerin olduğu piksellere 1 yaz
        out_shape=ref_shape,
        transform=ref_transform,
        fill=0, # Geri kalan pikseller 0 olsun
        dtype=rasterio.uint8
    )

    # Yeni raster'ı kaydet
    ref_meta.update(dtype='uint8', count=1, compress='lzw')
    with rasterio.open(output_path, 'w', **ref_meta) as dst:
        dst.write(rasterized_array, 1)

    print(f"'{output_path}' başarıyla oluşturuldu.")
    return rasterized_array

# Her katman için rasterleştirme işlemini yapalım
building_mask = rasterize_and_save(buildings, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'building_mask.tif'))
road_mask = rasterize_and_save(roads, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'road_mask.tif'))
green_space_mask = rasterize_and_save(green_spaces, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'green_space_mask.tif'))
water_mask = rasterize_and_save(water, REFERENCE_RASTER_PATH, os.path.join(PROCESSED_DIR, 'water_mask.tif'))

print("\nTüm OSM katmanları başarıyla rasterleştirildi ve kaydedildi.")

In [ ]:

# In[6]:
# --- Adım 4: Sonuçları Görselleştirme (Kontrol Amaçlı) ---
# Oluşturduğumuz raster maskelerden birini çizdirerek işlemin doğruluğunu kontrol edelim.

print("Bina maskesi (building_mask.tif) görselleştiriliyor...")

# Matplotlib ile görselleştirme
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(building_mask, cmap='viridis') # 1'ler sarı, 0'lar mor
ax.set_title('Rasterized Building Mask for Hamburg')
ax.set_axis_off() # Eksenleri kapat
plt.show()

In [ ]:
# In[7]:
# --- Özet ve Sonraki Adımlar ---
# Bu not defterinde, Hamburg için OSM verilerini indirip, modelimiz için gerekli olan
# 'binalar', 'yollar', 'yeşil alanlar' ve 'su' katmanlarını 10m çözünürlükte
# raster maskeler olarak hazırladık ve 'data/processed/' klasörüne kaydettik.
#
# Bir sonraki adım olan `03_data_processing.ipynb` not defterinde, bu maskeleri
# LST ve spektral indeksler (NDVI vb.) ile birleştirerek U-Net modelinin
# nihai girdisi olan çok kanallı tensörü oluşturacağız.